In [16]:
import smartsheet
from urllib.request import urlretrieve
import logging
import os
import shutil
import fnmatch

In [24]:
sheet_id = 1017210633932676

final_route = "tmp/final"
bank_route = "tmp/bank"
temp_route = "tmp/temp"
xlsx_name = "tmp/NAV Financial Report Import transactions_expenses_JRSHU_filename.xlsx"

cwd = os.getcwd()

# Ensure directories exist
os.makedirs(final_route, exist_ok=True)
os.makedirs(bank_route, exist_ok=True)
os.makedirs(temp_route, exist_ok=True)

# Check if bank route contains PDF files
receipt_pdf_files = [file for file in os.listdir(bank_route) if file.endswith(".pdf")]

if len(receipt_pdf_files) == 0:
    raise Exception("Bank route does not contain any PDF files.")

Exception: Bank route does not contain any PDF files.

In [18]:
# The API identifies columns by Id, but it's more convenient to refer to column names. Store a map here
import smartsheet.models
import smartsheet.sheets

column_map = {}


# Helper function to find cell in a row
def get_cell_by_column_name(
    row: smartsheet.models.Row, column_name: str
) -> smartsheet.models.Cell:
    column_id = column_map[column_name]
    return row.get_column(column_id)

In [19]:
smart = smartsheet.Smartsheet()
# Make sure we don't miss any error
smart.errors_as_exceptions(True)

# Log all calls
logging.basicConfig(filename="rwsheet.log", level=logging.INFO)

# Load entire sheet
sheet: smartsheet.models.sheet.Sheet = smart.Sheets.get_sheet(
    sheet_id="1017210633932676", include="attachments"
)

# Build column map for later reference - translates column names to column id
for column in sheet.columns:
    column_map[column.title] = column.id

In [20]:
class Attachment:
    def __init__(
        self,
        final_name: str,
        original_name: str,
        attachment: smartsheet.models.Attachment,
        download_url: str = None,
    ):
        self.final_name = final_name
        self.original_name = original_name
        self.attachment = attachment
        self.download_url = download_url

In [21]:
rows: list[smartsheet.models.row.Row] = sheet.rows
files: list[Attachment] = []


for row in rows:
    # print(row)
    op_number = get_cell_by_column_name(row, "OP elsz.sorszám").display_value
    ready_state = get_cell_by_column_name(row, "Ready")
    primary = get_cell_by_column_name(row, "Primary Column")
    receipt_number = get_cell_by_column_name(row, "Kifizetés bizonylata").display_value

    if ready_state.value != "Green":
        continue

    if receipt_number is not None and not receipt_number.startswith("P"):
        final_receipt_filename = f"{op_number}_b_{receipt_number}.pdf"
        found = False

        for receipt_file in receipt_pdf_files:
            if fnmatch.fnmatch(receipt_file, f"{receipt_number}_*.pdf"):
                # shutil.copy(f'{bank_route}/{receipt_file}', f'{temp_route}/{final_receipt_filename}')
                shutil.copy(
                    os.path.join(cwd, bank_route, receipt_file),
                    os.path.join(cwd, temp_route, final_receipt_filename),
                )
                found = True
                break

        print(f'Receipt "{receipt_number}" not found for OP {op_number}')

    for att in row.attachments:
        final_name = f"{op_number}__{att.name}"
        files.append(Attachment(final_name, att.name, att, download_url=None))

Receipt "258" not found for OP 011
Receipt "271" not found for OP 036
Receipt "326" not found for OP 288


In [22]:
for file in files:
    data: smartsheet.models.Attachment = smart.Attachments.get_attachment(
        sheet_id=sheet_id, attachment_id=file.attachment.id
    )
    file.download_url = data.url
    urlretrieve(data.url, os.path.join(cwd, temp_route, file.final_name))

In [23]:
from pypdf import PdfWriter

# Step 1: List all PDF files in the directory
receipt_pdf_files = [file for file in os.listdir(temp_route) if file.endswith(".pdf")]

# Step 2: Group files by the prefix before the underscore
pdf_groups: dict[str, list[str]] = {}

for file in receipt_pdf_files:
    prefix = file.split("_")[0]
    if prefix in pdf_groups:
        pdf_groups[prefix].append(file)
    else:
        pdf_groups[prefix] = [file]

# Step 3: Merge PDFs for each group
for prefix, files in pdf_groups.items():
    merger = PdfWriter()
    sorted_files = sorted(files)
    for file in files:
        merger.append(os.path.join(cwd, temp_route, file))
    output_filename = os.path.join(cwd, final_route, f"{prefix}_merged.pdf")
    merger.write(output_filename)
    merger.close()

In [27]:
import pandas as pd


df = pd.read_excel(os.path.join(cwd, xlsx_name))

In [28]:
df

,Country Code,Posting Date,Document No.,External Doc No,Account Type,Account No.,NAV Donor Buget No.,NAV Donor Budget Task No.,Description,Filename,Currency,Amount,Bal Acc Type,Bal Acc No,NAV Project,Department,Cat of interv,type of serv,Function,Comments
0,HUN,2024-02-01,1,V/0003/24,G/L Account,6012003,EUO-XNET,2A,Housing support_Horeb Ekpo_2024-01,HousingSupport202401,HUF,20000,Bank Account,HUN,HUN01,NaN,NaN,NaN,NaN,NaN
1,HUN,2024-01-02,2,V/0002/24,G/L Account,6012003,EUO-XNET,2A,Housing support_refugee_2024-01,HousingSupport202401,HUF,60000,Bank Account,HUN,HUN01,NaN,NaN,NaN,NaN,NaN
2,HUN,2024-01-03,3,V010004/24,G/L Account,6012003,EUO-XNET,2A,Housing support_Augustine Chinedu_2024-01,NaN,HUF,24000,Bank Account,HUN,HUN01,NaN,NaN,NaN,NaN,NaN
3,HUN,2024-01-03,4,V010003/24,G/L Account,6012003,EUO-XNET,2A,Housing support_Bada Gbolahan_2024-01,NaN,HUF,79000,Bank Account,HUN,HUN01,NaN,NaN,NaN,NaN,NaN
4,HUN,2024-01-03,5,V010002/24,G/L Account,6012003,EUO-XNET,2A,Housing support_Tar Nikolett_2024-01,NaN,HUF,59000,Bank Account,HUN,HUN01,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,HUN,2024-05-31,360,NaN,G/L Account,6012099,EUO-JUS,3A,Development teacher_Nyárádi Rita_2024-05,NaN,HUF,569520,Bank Account,HUN,HUN01,NaN,NaN,NaN,NaN,NaN
360,HUN,2024-05-31,361,ZÁ79,G/L Account,6012003,NaN,2D,Subsidies for medicines_Husakova Nina,NaN,HUF,13998,Bank Account,HUN,HUN01,NaN,NaN,NaN,NaN,NaN
361,HUN,2024-05-31,362,JMSZ/04/2024,G/L Account,6012099,NaN,2A,"1085 Bp., Mária u. 23., rental fee_2024.01-05",NaN,HUF,768716,Bank Account,HUN,HUN01,NaN,NaN,NaN,NaN,NaN
362,HUN,2024-05-31,363,JMSZ/03/2024,G/L Account,6012099,NaN,4A,Overhead - Printing cost_202301-05,NaN,HUF,192594,Bank Account,HUN,HUN01,NaN,NaN,NaN,NaN,NaN


In [29]:
final_files = os.listdir(os.path.join(cwd, final_route))

In [32]:
df.columns

Index(['Country Code', 'Posting Date', 'Document No.', 'External Doc No',
       'Account Type', 'Account No.', 'NAV Donor Buget No.',
       'NAV Donor Budget Task No.', 'Description', 'Filename', 'Currency',
       'Amount', 'Bal Acc Type', 'Bal Acc No', 'NAV Project', 'Department',
       'Cat of interv', 'type of serv', 'Function', 'Comments'],
      dtype='object')

In [37]:
for file in final_files:
    prefix = int(file.split("_")[0])
    if prefix not in df['Document No.'].values:
        print(f"OP {prefix} not found in the Excel file.")
        continue
    row = df.loc[df['Document No.'] == prefix]
    final_filename = f"{row['Country Code']}{str(row['Document No.']).zfill(3)}{row['Filename']}.pdf"
    shutil.copy(
        os.path.join(cwd, final_route, file),
        os.path.join(cwd, final_route, final_filename),
    )